In [1]:
import pandas as pd

In [2]:
comments_df = pd.read_csv("../komentarze.csv")

In [3]:
comments_df.tail()

,article,comment,is_positive,thumbs_up,thumbs_down,Unnamed: 5
1462,Klaudia Halejcio nagrywa córkę w... brudnej pi...,tak zwane selfi z idealną kupą. Patrzcie i po...,0,6,1,NaN
1463,Klaudia Halejcio nagrywa córkę w... brudnej pi...,Zabrakło tu wielu szczegółów pani Halejcio tzn...,0,13,2,NaN
1464,Klaudia Halejcio nagrywa córkę w... brudnej pi...,Ciekawe w której strefie pokoju zrobiło te kup...,0,13,0,NaN
1465,Klaudia Halejcio nagrywa córkę w... brudnej pi...,"Serio, serio ?. Wszystkim już siada od tego z...",0,8,0,NaN
1466,Klaudia Halejcio nagrywa córkę w... brudnej pi...,No to właśnie dziecku na przyszłość traume zap...,0,19,0,NaN


In [4]:
import emoji
def remove_emoji(string):
    string = emoji.get_emoji_regexp().sub("", string)
    return string

In [5]:
import string
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus.reader import pl196x
import nltk
from nltk.corpus import stopwords
import spacy 

def clean_text(text):
    # lower text
    text = text.lower()
    
    # remove emoji
    text = remove_emoji(text)

    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
   
    # remove stop words
    stops = get_stop_words('polish')
    words = [word for word in text if word not in stops]
    
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    text = " ".join(text)
    return(text)

comments_df["comments_clean"] = comments_df["comment"].apply(lambda x: clean_text(x))

In [6]:
comments_df["comments_clean"].head()

0    jakie to wszystko smutne plastikowy sztuczny ś...
1    haj girlsss howa areyou iam sooo goood fajansi...
2                    bomba dziewczyna buhaha wybuchłam
3                                           żałosne są
4                    kim ona jest ze ja ciagle promuja
Name: comments_clean, dtype: object

In [7]:
comments_df["is_positive"].value_counts()

0     1305
1      161
)        1
Name: is_positive, dtype: int64

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
dv = vectorizer.fit_transform(comments_df["comments_clean"])

In [9]:
# vectorizer.get_feature_names()

In [10]:
dshuf = comments_df.sample(frac = 1)

In [11]:
dtrain = dshuf[:700]
dtest = dshuf[700:]
dtrain_att = vectorizer.fit_transform(dtrain['comments_clean'])
dtest_att = vectorizer.transform(dtest['comments_clean'])
dtrain_label = dtrain['is_positive']
dtest_label = dtest['is_positive']

In [12]:
dtest.count()

article           767
comment           767
is_positive       767
thumbs_up         767
thumbs_down       767
Unnamed: 5          2
comments_clean    767
dtype: int64

In [13]:
dtrain.count()

article           700
comment           700
is_positive       700
thumbs_up         700
thumbs_down       700
Unnamed: 5          1
comments_clean    700
dtype: int64

In [14]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators = 50, random_state = 0)
RFC.fit(dtrain_att, dtrain_label)

RandomForestClassifier(n_estimators=50, random_state=0)

In [15]:
print(comments_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1467 entries, 0 to 1466
Data columns (total 7 columns):
article           1467 non-null object
comment           1467 non-null object
is_positive       1467 non-null object
thumbs_up         1467 non-null int64
thumbs_down       1467 non-null int64
Unnamed: 5        3 non-null float64
comments_clean    1467 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 80.4+ KB
None


In [16]:
RFC.score(dtrain_att, dtrain_label)

0.9985714285714286

In [17]:
from sklearn.metrics import confusion_matrix
y_pred = RFC.predict(dtrain_att)
confusion_matrix(y_pred, dtrain_label)

array([[626,   1],
       [  0,  73]])

In [18]:
from sklearn.pipeline import Pipeline,make_pipeline
pl = Pipeline([
 ('bag of words: ', CountVectorizer()),
 ('Random Forest Classifier:', RandomForestClassifier())])
make_pipeline(CountVectorizer(), RandomForestClassifier())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [19]:
dshuf = comments_df.sample(frac=1)
d_content = dshuf['comments_clean']
d_label = dshuf['is_positive']

In [20]:
d_label[:149]

1406    0
374     0
332     0
643     0
1455    0
       ..
277     0
429     0
483     0
1361    1
1369    0
Name: is_positive, Length: 149, dtype: object

In [21]:
d_content[:149]

1406                            dostala spadek to ja stac
374                              niech się cieszy że zyje
332                          a co jej tak psychika siadła
643                                      tvn i wole media
1455    chora tylko jej dziecko robi kupe nie ludzie s...
                              ...                        
277     ale julka była pewna że ten barto ją wybierze ...
429                                           karma wraca
483                skad on w ogole ma kasę na to wszystko
1361                aniu kochamy cie ja też ciebie kocham
1369    bardzo dobrze ze nie jest juz jurorka nie nada...
Name: comments_clean, Length: 149, dtype: object

In [22]:
pl.fit(d_content[:1466],d_label[:1466])

Pipeline(steps=[('bag of words: ', CountVectorizer()),
                ('Random Forest Classifier:', RandomForestClassifier())])

In [23]:
pl.score(d_content[:1466],d_label[:1466])

1.0

In [24]:
pl.predict(["jest słaba, przestańcie o niej pisać"])

array(['0'], dtype=object)

In [25]:
pl.predict(["lubię ją! super babka."])

array(['1'], dtype=object)

In [26]:
pl.predict(["super jest! uwielbiam ją!"])

array(['1'], dtype=object)

In [27]:
pl.predict(["jest śliczna! super wygląda"])

array(['1'], dtype=object)

In [28]:
pl.predict(["żałosna, nie lubię jej."])

array(['0'], dtype=object)

In [29]:
pl.predict(["wygląda ślicznie. ma super figurę. widać, że jest szczęśliwa!"])

array(['1'], dtype=object)

In [30]:
pl.predict(["wygląda staro, nie mogę na nią patrzeć"])

array(['0'], dtype=object)

In [31]:
pl.predict(["Gratulacje! Będą mieli śliczne dzieci!"])

array(['1'], dtype=object)

In [32]:
pl.predict(["Nie mogę na nich patrzeć, okropnie wygląda w tym kostiumie!"])

array(['0'], dtype=object)

In [33]:
pl.predict(["Wreszcie normalna babka!"])

array(['1'], dtype=object)

In [34]:
pl.predict(["Ile można o nich pisać?"])

array(['0'], dtype=object)